In [ ]:
pip install fingerflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
RADIUS = 5

from datetime import datetime
import cv2
import numpy as np
import matplotlib.pyplot as plt
from fingerflow.extractor import Extractor
def get_minutiae_color(point_class):
    switcher = {
        0: (0, 0, 255),
        1: (0, 255, 0),
        2: (255, 0, 0),
        3: (0, 255, 255),
        4: (255, 255, ),
        5: (255, 0, 255)
    }

    return switcher.get(point_class)


current_datetime = datetime.now().strftime("%d%m%Y%H%M%S")

extractor = Extractor("/content/drive/MyDrive/CoarseNet.h5", "/content/drive/MyDrive/FineNet.h5", "/content/drive/MyDrive/ClassifyNet_6_classes.h5", "/content/drive/MyDrive/CoreNet.weights")

image = cv2.imread("/content/drive/MyDrive/Data Collection/F_L1.jpg")

extracted_data = extractor.extract_minutiae(image)

extracted_minutiae = extracted_data['minutiae']
core = extracted_data['core']

start_point = (core['x1'].values[0], core['y1'].values[0])
end_point = (core['x2'].values[0], core['y2'].values[0])
color = (0, 0, 255)

cv2.rectangle(image, start_point, end_point, color, 2)


if len(extracted_minutiae) > 0:
    for index, minutia in extracted_minutiae.iterrows():
        print(minutia)
        x = minutia['x']
        y = minutia['y']
        o = minutia['angle']
        minutia_class = minutia['class']

        start_point = (int(x-RADIUS), int(y-RADIUS))
        end_point = (int(x+RADIUS), int(y+RADIUS))

        color = get_minutiae_color(int(minutia_class))

        cv2.rectangle(image, start_point, end_point, color, 2)

        line_start_point = (int(x), int(y))
        line_end_point = (int(x + 20 * np.cos(o)), int(y + 20 * np.sin(o)))

        cv2.line(image, line_start_point, line_end_point, color, 2)

cv2.imwrite(f'extractor_single_img_test_{current_datetime}.png', image)

Loading FineNet weights from /content/drive/MyDrive/FineNet.h5
Loading ClassifyNet weights from /content/drive/MyDrive/ClassifyNet_6_classes.h5
nms iou: 0.413 score: 0.3
all weights read
Core net weights loaded from /content/drive/MyDrive/CoreNet.weights


/usr/local/lib/python3.10/dist-packages/fingerflow/extractor/MinutiaeNet/CoarseNet/minutiae_net_utils.py:660: FutureWarning: `multichannel` is a deprecated argument name for `gaussian`. It will be removed in version 1.0. Please use `channel_axis` instead.
  cos_2_theta = filters.gaussian(cos_2_theta, sigma, multichannel=False, mode='reflect')
/usr/local/lib/python3.10/dist-packages/fingerflow/extractor/MinutiaeNet/CoarseNet/minutiae_net_utils.py:661: FutureWarning: `multichannel` is a deprecated argument name for `gaussian`. It will be removed in version 1.0. Please use `channel_axis` instead.
  sin_2_theta = filters.gaussian(sin_2_theta, sigma, multichannel=False, mode='reflect')


1/1 [==============================] - 4s 4s/step
x        340.000000
y        605.000000
angle      1.694035
score      0.799767
class      4.000000
Name: 0, dtype: float64
x        261.000000
y        500.000000
angle      5.343433
score      0.799471
class      4.000000
Name: 1, dtype: float64
x        276.000000
y        454.000000
angle      6.209919
score      0.799416
class      4.000000
Name: 2, dtype: float64
x        277.000000
y        603.000000
angle      1.857962
score      0.799273
class      4.000000
Name: 3, dtype: float64
x        286.000000
y        553.000000
angle      1.776870
score      0.799144
class      4.000000
Name: 4, dtype: float64
x        331.000000
y        532.000000
angle      1.380199
score      0.798866
class      4.000000
Name: 5, dtype: float64
x        338.000000
y        635.000000
angle      1.726655
score      0.797040
class      4.000000
Name: 6, dtype: float64
x        289.000000
y        475.000000
angle      0.793483
score      0.793187
cl

True

In [ ]:
import pandas as pd
MINUTIAE_NUM = 20

extractor = Extractor("/content/drive/MyDrive/CoarseNet.h5", "/content/drive/MyDrive/FineNet.h5", "/content/drive/MyDrive/ClassifyNet_6_classes.h5", "/content/drive/MyDrive/CoreNet.weights")

image = cv2.imread("/content/drive/MyDrive/Data Collection/F_L1.jpg")

extracted_minutiae = extractor.extract_minutiae(image)


def get_correct_core_point(core_data):
    correct_core = core_data[core_data.score == core_data.score.max()]

    x = correct_core[['x1', 'x2']].mean(axis=1)
    y = correct_core[['y1', 'y2']].mean(axis=1)

    return pd.DataFrame({'x': x.values, 'y': y.values})


def get_n_nearest_minutiae(extracted_data, core_point):
    minutiae_data = extracted_data['minutiae']

    if len(core_point) == 0 or len(minutiae_data) < MINUTIAE_NUM:
        return pd.DataFrame()

    minutiae_data['core_distance'] = np.linalg.norm(
        minutiae_data[['x', 'y']].values - core_point.values, axis=1)

    return minutiae_data.sort_values(by=['core_distance']).reset_index(drop=True)


def get_minutiae_color(point_class):
    switcher = {
        0: (255, 255, 255),
        1: (0, 255, 0),
        2: (255, 0, 0),
        3: (0, 255, 255),
        4: (255, 255, ),
        5: (255, 0, 255)
    }

    return switcher.get(point_class)


def show_core_point(core):
    start_point = (core['x1'].values[0], core['y1'].values[0])
    end_point = (core['x2'].values[0], core['y2'].values[0])
    color = (0, 0, 255)

    cv2.rectangle(image, start_point, end_point, color, 2)


def show_minutiae_points(minutiae, core_point):
    radius = 10
    core = (int(core_point['x'].values[0]), int(core_point['y'].values[0]))

    for index, point in minutiae.iterrows():
        center = (int(point['x']), int(point['y']))

        print(center, index)
        cv2.circle(image, center, radius, get_minutiae_color(int(point['class'])), 3)

        if index < MINUTIAE_NUM:
            cv2.line(image, core, center, (0, 0, 255), 1)


core_point = get_correct_core_point(extracted_minutiae['core'])
enriched_minutiae = get_n_nearest_minutiae(extracted_minutiae, core_point)

show_core_point(extracted_minutiae['core'])
show_minutiae_points(enriched_minutiae, core_point)

print(enriched_minutiae.to_markdown())

cv2.imwrite('visualised_feature_vector_3.png', image)
